Admin Scripts for SQL Security. First we create our demo user the reckless programmer Broseph.

In [1]:
USE [master]
GO
CREATE LOGIN [Broseph] WITH PASSWORD=N'yolo', DEFAULT_DATABASE=[BP_BabbyNames], CHECK_EXPIRATION=OFF, CHECK_POLICY=OFF
GO

USE [BP_BabbyNames]
GO
CREATE USER [Broseph] FOR LOGIN [Broseph]
GO
ALTER ROLE [db_owner] ADD MEMBER [Broseph]
GO
ALTER ROLE [db_datawriter] ADD MEMBER [Broseph]
GO
ALTER ROLE [db_datareader] ADD MEMBER [Broseph]
GO


Commands completed successfully.

Mail (Id: 15) queued.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.087

## Server Level Permissions

Custom server roles are not terribly common. Generally if you actually need server wide permissions you end up with sys admin becuase you manage the instance. We do have a use case where we wanted to allow people permissions to help troubleshoot without actually opening up the server to them. The following role will allow a user to see server state (i.e. use whoisactive and access DMVs as well as see the definition of any object on the server). Uses in this role won't be able to alter any server settings or access any data unless they have specific permissions for those resources granted from other roles.

In [3]:
USE [master];
GO

CREATE SERVER ROLE [crole_DBDeveloper]
GO

GRANT VIEW SERVER STATE TO [crole_DBDeveloper]
GO

GRANT CONNECT ANY DATABASE TO [crole_DBDeveloper]
GO

GRANT VIEW ANY DEFINITION TO [crole_DBDeveloper]
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.127

In [5]:
ALTER SERVER ROLE [crole_DBDeveloper] ADD MEMBER [Broseph]
GO

Commands completed successfully.

Total execution time: 00:00:00.026

In [4]:
USE master

Commands completed successfully.

Total execution time: 00:00:00.010